# Load libraries

In [13]:
import os
from pathlib import Path
import geopandas as gpd
import pandas as pd

# STEP 0: Make data folder

In [14]:
### create data directory
data_dir = os.path.join(
    
    ### home directory
    pathlib.Path.home(),

    ### make folders
    'earth-analytics',
    'data',

    ### project directory for this assignment
    'vegetation'
)

## make the path
os.makedirs(data_dir, exist_ok = True)

# Step 1: Get vector data for Gila Reservation
A note on terminology: We realize not everyone is cool with the term "American Indian," especially when used by people who aren't Tribal members. At the same time, others feel that it is important to reclaim "Indian," and more importantly that's the term a lot of Tribal people use every day. In this case, we're stuck because of how this dataset is named in the Census boundary datasets...however, when sharing your work or publishing your own data, it's important to include local communities in teh area you're studying, in language choices but also _throughout_ your project.

We'll be downloading the boundaries of our study area using a web browser. The data source, data.gov, has placed restrictions on programmatic downloads (downloads using code). There are a number of ways to attempt to get around these restrictions, but in this case we've found it's just not worth the time and effort. Follow the steps below to download the data and put it somewhere that your Python code can find it.

## Step 1A: Open up the data catalog
Click this link: https://catalog.data.gov. You can also open your web browser and type in the url if you prefer.

## Step 1B: Search for the dataset
In the search bar, search for "American Indian Tribal Subdivisions 2020." These census files are updated regularly at the time of the census, so we want to make sure we have the most recent version. The most recent census was in 2020.

## Step 1C: Open the dataset page
Find the “TIGER/Line Shapefile, 2020, Nation, U.S., American Indian Tribal Subdivisions” dataset in the search results, and click on the title to go to the dataset page. 

## Step 1D: Download
Next, scroll down to the available files to download. Click on the Download button for the .zip file -- this will be the easiest one for Python to open.

## Step 1E: Move the file
We now need to locate the file you downloaded and put it somewhere Python can find it. Ideally, you should put the downloaded .zip file in your project directory. Most web browsers will pop up with some kind of button to open your File Explorer (Windows) or Finder (Mac) in the location of your downloaded files. You can also check in your user home directory for a Downloads folder. If none of that works, try opening up your File Explorer/Finder and searching for the file.

First, you'll need to get the filepath for the downloaded data:

In [15]:
### get the file path for the downloaded zip file

### make a file path for your downloads folder
downloads = os.path.join(os.path.expanduser("~"), "Downloads")

### make a placeholder (empty) source path to save the file path into
source_path = None  

### find the zip file
for f in os.listdir(downloads):
    if "tl_2020_us_aitsn" in f.lower():
        source_path = os.path.join(downloads, f)
        print("Found file:", source_path)

        ### use "break" to stop after you find the first file that matches
        break  

source_path

Found file: C:\Users\kjsie\Downloads\tl_2020_us_aitsn.zip


'C:\\Users\\kjsie\\Downloads\\tl_2020_us_aitsn.zip'

Then you can use code to copy the zip file into your data directory:

In [10]:
### open shutil package
import shutil

### copy the file to your data directory
shutil.copy(source_path, data_dir)

'C:\\Users\\kjsie\\earth-analytics\\data\\vegetation\\tl_2020_us_aitsn.zip'

## Step 1F: Unzip the file (optional)

You can optionally unzip the file. `geopandas` will be able to read this particular shapefile from a `.zip` archive, but some other files may need to be unzipped. We’ll first define the path in Python, and then use bash to unzip. You can also unzip with the `Python` `zipfile` library, but it is more complicated than we need right now.

In [16]:
### identify file name
filename = "tl_2020_us_aitsn"

### identify location of zipped file
zip_path = Path(data_dir) / f"{filename}.zip"

### identify where you want the unzipped files to go
unzip_dir = Path(data_dir) / filename

Now we can unzip the file using the package zipfile. 

Nate's demo uses bash commands, but I'm switching to use the zipfile package because it's interoperable across operating systems.

In [18]:
### load zipfile package
import zipfile

### make sure paths are all Path objects
zip_path = Path(zip_path)
unzip_dir = Path(unzip_dir)

### unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

## Step 1E: Open boundary in Python

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>"directory-name"</code> with the actual name of the
file you downloaded (or the folder you unzipped to) and put in your
project directory.</li>
<li>Modify the code below to use <strong>descriptive variable
names</strong>. Feel free to refer back to previous challenges for
similar code!</li>
<li>Add a line of code to open up the data path. What library and
function do you need to open this type of data?</li>
<li>Add some code to check your data, either by viewing it or making a
quick plot. Does it look like what you expected?</li>
</ol></div></div>

In [23]:
### define boundary path (we've already done this when we unzipped the file)
unzip_dir

### open the site boundary
aitsn_boundary = gpd.read_file(unzip_dir)

### check that the data were downloaded correctly
aitsn_boundary

,AIANNHCE,TRSUBCE,TRSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,2430,653,02419073,2430653,Red Valley,Red Valley Chapter,T2,D7,G2300,A,922036695,195247,+36.6294607,-109.0550394,"POLYGON ((-109.2827 36.64644, -109.28181 36.65..."
1,2430,665,02419077,2430665,Rock Point,Rock Point Chapter,T2,D7,G2300,A,720360268,88806,+36.6598701,-109.6166836,"POLYGON ((-109.85922 36.49859, -109.85521 36.5..."
2,2430,675,02419081,2430675,Rough Rock,Rough Rock Chapter,T2,D7,G2300,A,364475668,216144,+36.3976971,-109.7695183,"POLYGON ((-109.93053 36.40672, -109.92923 36.4..."
3,2430,325,02418975,2430325,Indian Wells,Indian Wells Chapter,T2,D7,G2300,A,717835323,133795,+35.3248534,-110.0855000,"POLYGON ((-110.24222 35.36327, -110.24215 35.3..."
4,2430,355,02418983,2430355,Kayenta,Kayenta Chapter,T2,D7,G2300,A,1419241065,1982848,+36.6884391,-110.3045616,"POLYGON ((-110.56817 36.73489, -110.56603 36.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,1310,100,02418907,1310100,1,District 1,28,D7,G2300,N,139902197,0,+33.0600842,-111.5806313,"POLYGON ((-111.63622 33.11798, -111.63405 33.1..."
480,4290,550,02612186,4290550,Mission Highlands,Mission Highlands,00,D7,G2300,N,6188043,0,+48.0754384,-122.2507432,"POLYGON ((-122.27579 48.07128, -122.27578 48.0..."
481,0855,400,02418941,0855400,Fort Thompson,Fort Thompson District,07,D7,G2300,N,535432708,38653364,+44.1559680,-099.4467700,"POLYGON ((-99.66452 44.25269, -99.66449 44.255..."
482,0335,300,02784108,0335300,Indian Point,Indian Point Segment,T3,D7,G2300,N,326985,0,+48.0604594,-092.8466753,"POLYGON ((-92.85187 48.05944, -92.85186 48.059..."


# Step 2: Pull out the Gila River subdivisions

We'll use the AIANNHCE code for the Gila River (1310).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>identifier</code> with the value you found from
exploring the interactive map. Make sure that you are using the correct
<strong>data type</strong>!</li>
<li>Change the plot to have a web tile basemap, and look the way you
want it to.</li>
</ol></div></div>

In [26]:
# Select and merge the subdivisions you want

### subset aitsn_boundary to just include the Gila River locations
### use dissolve to combine multiple geometries into a single polygon/multipolygon
gila_gdf = aitsn_boundary[aitsn_boundary["AIANNHCE"] == "1310"].dissolve()
gila_gdf

# Plot the results with web tile images

### import hvplot
import hvplot.pandas

### plot the study area
gila_gdf.hvplot(geo = True, tiles = 'EsriImagery',
                fill_color = None, line_color = 'black',
                title = 'Gila River Indian Community',
                frame_width = 500)

c:\Users\kjsie\anaconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

## Alternative workflow: open the boundary data without downloading it
Here, you will open the data in Python without downloading it. You will need the link to the zip file download. Follow steps 1A-1C to get to the dataset page, then right-click on the "Download" button next to the zip file and copy the link.

In [27]:
### create url for AITSN data from data.gov
aitsn_url = "https://www2.census.gov/geo/tiger/TIGER2020/AITSN/tl_2020_us_aitsn.zip"
aitsn_url

'https://www2.census.gov/geo/tiger/TIGER2020/AITSN/tl_2020_us_aitsn.zip'

In [ ]:
### open AITSN URL with geopandas
aitsn = gpd.read_file(aitsn_url)
aitsn

# Step -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [30]:
%store gila_gdf aitsn_boundary

Stored 'gila_gdf' (GeoDataFrame)
Stored 'aitsn_boundary' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!